In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import gc

In [3]:
main_df = pd.read_csv('main_df.csv')
fea_col = sorted(list(set(list(main_df.columns)) - set(['matrl_id','year'])))

In [4]:
main_df.columns

Index(['matrl_id', 'year', 'month', 'month_sale', 'host_nums_01',
       'host_nums_02', 'host_nums_03', 'host_nums_04', 'host_nums_05',
       'host_nums_07', 'host_nums_08', 'host_nums_09', 'host_nums_10',
       'host_nums_13', 'host_nums_18', 'host_nums_19', 'host_nums_32',
       'host_nums_33', 'host_nums_34', 'host_nums_35', 'host_nums_36',
       'host_nums_A7', 'work_time_01', 'work_time_02', 'work_time_03',
       'work_time_04', 'work_time_05', 'work_time_33', 'work_time_34',
       'work_time_35', 'work_time_A7', 'work_nums_01', 'work_nums_02',
       'work_nums_03', 'work_nums_04', 'work_nums_05', 'work_nums_33',
       'work_nums_34', 'work_nums_35', 'work_nums_A7'],
      dtype='object')

In [5]:
diff_col = sorted(list(set(list(main_df.columns)) - set(['matrl_id','year','month'])))

In [12]:
for col in diff_col:
    main_df[f'diff_{col}'] = main_df.groupby(['matrl_id'])[col].diff(1)

In [52]:
fea_col = sorted(list(set(list(main_df.columns)) - set(['matrl_id','year'])))

In [70]:
def series_to_superviesed(x_timeseries, y_timeseries, n_memory_step, n_forcast_step, split = None):
    input_step, input_feature = x_timeseries.shape
    output_step, output_feature = y_timeseries.shape

    n_RNN_sample=input_step-n_forcast_step-n_memory_step+1
    RNN_x=np.zeros((n_RNN_sample,n_memory_step, input_feature))
    RNN_y=np.zeros((n_RNN_sample,n_forcast_step, output_feature))

    for n in range(n_RNN_sample):
        RNN_x[n,:,:]=x_timeseries[n:n+n_memory_step,:]
        RNN_y[n,:,:]=y_timeseries[n+n_memory_step:n+n_memory_step+n_forcast_step,:]
    if split != None:
        assert (split <=0.9) & (split >= 0.1), 'split not in reasonable range'
        return RNN_x[:int(split*len(RNN_x))], RNN_y[:int(split*len(RNN_x))],\
               RNN_x[int(split*len(RNN_x))+1:], RNN_y[int(split*len(RNN_x))+1:]
    else:
        return RNN_x, RNN_y, None, None

X, Y = [], []
X_test = []
for i in range(1,1201):
    df_i = main_df[main_df['matrl_id']==i].copy()
    x_timeseries = df_i[fea_col].values
    y_timeseries = df_i["month_sale"].values.reshape(-1,1)
    x_train,y_train,x_test,y_test = series_to_superviesed(x_timeseries,y_timeseries,3,3)
    if i == 1:
        print(x_train.shape)
        print(y_train.shape)
    X.append(x_train)
    Y.append(y_train)
    x_test = df_i[fea_col].values[-3:].reshape(1,3,75)
    X_test.append(x_test)
    del x_train,y_train,x_test,y_test,df_i
    gc.collect()

(25, 3, 75)
(25, 3, 1)


In [71]:
X = np.vstack(X)
Y = np.vstack(Y)
X = X.reshape(-1,3*75)
Y = Y.reshape(-1,3)
print(X.shape)

(30000, 225)


In [73]:
X_test = np.vstack(X_test)
X_test = X_test.reshape(-1,3*75)

In [74]:
X_test.shape

(1200, 225)

In [56]:
test_idx = [1]*1200 + [2]*1200 + [3]*1200
test_idx = np.array(test_idx).reshape(-1,1)
X_test_new = np.vstack([X_test,X_test,X_test])
X_test_new = np.hstack([X_test_new,test_idx])

In [57]:
X_test_new.shape

(3600, 901)

In [58]:
idx = [1]*19200 + [2]*19200 + [3]*19200
idx = np.array(idx).reshape(-1,1)

In [59]:
X_new = np.vstack([X,X,X])
X_new = np.hstack([X_new,idx])

In [60]:
Y_new = np.hstack([Y[:,0],Y[:,1],Y[:,2]])
print(Y_new.shape)

(57600,)


In [75]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

In [77]:
skf = KFold(n_splits=5)
fold = 0
for train_index, valid_index in skf.split(X):
    
    for y_id in range(3):
        model = LGBMRegressor(boosting_type='gbdt', objective='regression', num_leaves=128,
                              learning_rate=0.01, n_estimators=10000, max_depth=-1,
                              metric='rmse', subsample=0.8, subsample_freq = 5,
                              colsample_bytree=0.8,
                              reg_alpha=0.1, reg_lambda=0.2, random_state = 2021)
        model.fit(
            X[train_index], Y[train_index,y_id],
            eval_set=[(X[valid_index], Y[valid_index,y_id])],
            eval_metric='rmse',
            early_stopping_rounds=50,
            verbose=100
        )
    
        if fold == 0:
            if y_id == 0:
                pred0 = model.predict(X_test)/5
            elif y_id == 1:
                pred1 = model.predict(X_test)/5
            else:
                pred2 = model.predict(X_test)/5
        else:
            if y_id == 0:
                pred0 += model.predict(X_test)/5
            elif y_id == 1:
                pred1 += model.predict(X_test)/5
            else:
                pred2 += model.predict(X_test)/5
    fold += 1

C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's rmse: 77.8347
[200]	valid_0's rmse: 70.2586
[300]	valid_0's rmse: 70.3288


C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's rmse: 71.9101
[200]	valid_0's rmse: 62.5146


C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

[100]	valid_0's rmse: 103.025
[200]	valid_0's rmse: 96.9648


C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's rmse: 120.575


C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's rmse: 118.81


In [83]:
pred0

array([ 5.3992389 , 19.9473084 ,  8.83060649, ...,  4.89364751,
       20.51537794,  5.08771659])

In [84]:
sub = pd.DataFrame(columns=["matrl_id","month","nums"])
for i in range(1,1201):
    temp = pd.DataFrame({"matrl_id":[i,i,i],"month":[202007,202008,202009],"nums":[round(pred0[i-1]),round(pred1[i-1]),round(pred2[i-1])]})
    sub=sub.append(temp,ignore_index=True)
    

In [85]:
sub.to_csv('lgb.csv',index=None)

In [86]:
sub

,matrl_id,month,nums
0,1,202007,5
1,1,202008,7
2,1,202009,11
3,2,202007,20
4,2,202008,20
...,...,...,...
3595,1199,202008,30
3596,1199,202009,34
3597,1200,202007,5
3598,1200,202008,6
